In [4]:
import pandas as pd
from pyvis.network import Network
import networkx as nx

# load csv file
csv_file = "data.csv"
df = pd.read_csv(csv_file)

# replace NaN in columns with empty strings
df['notes'] = df['notes'].fillna("")
df['connection_labels'] = df['connection_labels'].fillna("")
df['connection_directions'] = df['connection_directions'].fillna("")

# initialise the pyvis network
net = Network(notebook=False, directed=True, cdn_resources="in_line")

# add nodes with colours for albums, artists, loved items, need to listen
for _, row in df.iterrows():
    # assign colours based on type and love
    if row['type'] == "album":
        if row['to_listen'] == 1:
            color = "AliceBlue"  # pale soft blue for need to listen
        else:
            color = "#6495ED" if not row['love'] == 1 else "#4263B4"  # dark cornflower blue for loved albums
    elif row['type'] == "artist":
        color = "SandyBrown" if not row['love'] == 1 else "DarkOrange"
    else:
        color = "gray"  # grey default colour if type is not recognised

    # add node
    net.add_node(
        row['item_id'],
        label=row['item_name'],
        title=row['notes'],
        color=color
    )

# add directional edges with labels
for _, row in df.iterrows():
    if pd.notna(row['connections']):
        connections = row['connections'].split('|')
        labels = row['connection_labels'].split('|') if row['connection_labels'] else [None] * len(connections)
        directions = row['connection_directions'].split('|') if row['connection_directions'] else ["none"] * len(connections)

        for conn, label, direction in zip(connections, labels, directions):
            arrow_to = True if direction == "forward" else False
            arrow_from = True if direction == "backward" else False
            net.add_edge(
                row['item_id'], int(conn),
                title = label,
                arrowStrikethrough = False,
                arrows = {'to': arrow_to, 'from': arrow_from}
            )

# save html output
net.set_options('''
{
  "layout": {
    "improvedLayout": true
  },
  "interaction": {
    "zoomView": true,
    "dragView": true,
    "hover": true,
    "tooltipDelay": 300
  },
  "physics": {
    "stabilization": {
      "enabled": true
    }
  }
}
''')

output_file = "index.html"
net.save_graph(output_file)

# add responsive design meta tag and styles to the html
legend_html = """
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        body, html {
            margin: 0;
            padding: 0;
            width: 100%;
            height: 100%;
            overflow: hidden;
        }
        #mynetwork {
            width: 100%;
            height: calc(100vh - 80px); /* Adjust height to leave space for the legend */
        }
        .legend {
            position: absolute;
            bottom: 0;
            left: 0;
            right: 0;
            background: white;
            padding: 10px 5px;
            font-family: Arial, sans-serif;
            font-size: 1em; /* Responsive font size */
            border-top: 1px solid black;
            text-align: center;
            display: flex;
            flex-wrap: wrap; /* Allow items to wrap on smaller screens */
            justify-content: center;
            gap: 10px; /* Add spacing between items */
        }
        .legend-item {
            display: flex;
            align-items: center;
            margin: 5px;
        }
        .legend-color {
            display: inline-block;
            width: 15px;
            height: 15px;
            margin-right: 5px;
            border-radius: 50%;
        }
    </style>
</head>
"""

# add custom javascript for click event handling
click_script = """
<script>
    var infoBox = document.createElement("div");
    infoBox.id = "node-info";
    infoBox.style.position = "fixed";
    infoBox.style.top = "10px"; /* Move to top */
    infoBox.style.left = "10px"; /* Adjust horizontal position */
    infoBox.style.background = "white";
    infoBox.style.padding = "10px";
    infoBox.style.border = "1px solid black";
    infoBox.style.display = "none";
    infoBox.style.zIndex = "1000"; /* Ensure it appears above other elements */
    infoBox.style.fontFamily = "Arial, sans-serif";
    infoBox.style.fontSize = "14px";
    document.body.appendChild(infoBox);

    network.on("click", function(params) {
        if (params.nodes.length > 0) {
            // Display node information
            var nodeId = params.nodes[0];
            var nodeData = network.body.data.nodes.get(nodeId);
            infoBox.innerHTML = nodeData.title;
            infoBox.style.display = "block";
        } else if (params.edges.length > 0) {
            // Display edge information
            var edgeId = params.edges[0];
            var edgeData = network.body.data.edges.get(edgeId);
            infoBox.innerHTML = (edgeData.title || "No info");
            infoBox.style.display = "block";
        } else {
            infoBox.style.display = "none";
        }
    });
</script>
"""

# insert the script into the saved html
with open(output_file, "r") as file:
    html_content = file.read()

html_content = html_content.replace("</body>", click_script + "</body>")

with open(output_file, "w") as file:
    file.write(html_content)

legend_html += """
<div class="legend">
    <div class="legend-item"><span class="legend-color" style="background-color: #6495ED;"></span>Album</div>
    <div class="legend-item"><span class="legend-color" style="background-color: #4263B4;"></span>Favourite album</div>
    <div class="legend-item"><span class="legend-color" style="background-color: AliceBlue;"></span>Album to listen</div>
    <div class="legend-item"><span class="legend-color" style="background-color: SandyBrown;"></span>Artist</div>
    <div class="legend-item"><span class="legend-color" style="background-color: DarkOrange;"></span>Favourite artist</div>
</div>
"""

# insert the legend and meta tag into the saved html file
with open(output_file, "r") as file:
    html_content = file.read()

html_content = html_content.replace("<head>", legend_html + "<head>")
html_content = html_content.replace("</body>", "</body>")

with open(output_file, "w") as file:
    file.write(html_content)

print(f"Interactive network with legend saved as {output_file}.")

Interactive network with legend saved as index.html.
